# Lesson 1: Your First AI Agent System

**Goal:** Build a complete working AI system in 2 hours

**What you'll learn:**
- How to call local LLMs (LM Studio)
- Structured outputs with Pydantic
- Building AI agents that make decisions
- Multi-agent orchestration

**Fast.ai approach:** Start with working code, understand by experimenting

---

## Prerequisites Check

✅ LM Studio running on localhost:1234  
✅ Llama 3.1 8B Instruct model loaded  
✅ Python packages installed (httpx, pydantic)

In [ ]:
# Test LM Studio connection
import httpx

try:
    response = httpx.get("http://localhost:1234/v1/models")
    print("✅ LM Studio is running!")
    print(f"Model: {response.json()['data'][0]['id']}")
except Exception as e:
    print("❌ LM Studio not running. Start it first!")
    print(f"Error: {e}")

---

## Part 1: First LLM Call

Let's start simple: send text, get text back.

In [ ]:
import asyncio
import httpx

async def simple_llm_call(prompt: str) -> str:
    """Simplest possible LLM call"""
    async with httpx.AsyncClient(timeout=30.0) as client:
        response = await client.post(
            "http://localhost:1234/v1/chat/completions",
            json={
                "model": "llama-3.1-8b-instruct",
                "messages": [{"role": "user", "content": prompt}],
                "temperature": 0.7,
            }
        )
        result = response.json()
        return result["choices"][0]["message"]["content"]

# Test it
response = await simple_llm_call("Say hello in exactly 5 words")
print(f"LLM: {response}")

**🎓 Exercise:** Change the prompt and run again. Try:
- "Count from 1 to 10"
- "Explain AI in one sentence"
- "Write a haiku about code"

---

## Part 2: Structured Outputs

**The Secret Sauce:** Make LLMs return structured data

This is THE most important pattern in LLM engineering.

In [ ]:
from pydantic import BaseModel, Field
import json

# Define output structure
class Analysis(BaseModel):
    is_relevant: bool = Field(description="Is this relevant?")
    confidence: float = Field(description="Confidence 0-1", ge=0, le=1)
    reasoning: str = Field(description="Why?")

# Show schema
print("Schema:")
print(json.dumps(Analysis.model_json_schema(), indent=2))

In [ ]:
async def get_structured(prompt: str, schema: BaseModel) -> BaseModel:
    """Get LLM output matching schema"""
    
    # Add schema to prompt
    schema_json = json.dumps(schema.model_json_schema(), indent=2)
    full_prompt = f"""{prompt}

Respond with valid JSON matching this schema:
{schema_json}

Rules: ONLY JSON, no markdown, no explanations"""
    
    # Call LLM
    response = await simple_llm_call(full_prompt)
    
    # Clean and parse
    cleaned = response.strip().replace("```json", "").replace("```", "").strip()
    return schema.model_validate_json(cleaned)

# Test
result = await get_structured(
    "Analyze: 'AI Cybersecurity Platform Development'",
    Analysis
)

print("\n✅ Structured Output:")
print(f"Relevant: {result.is_relevant}")
print(f"Confidence: {result.confidence}")
print(f"Reasoning: {result.reasoning}")

**🎓 What's powerful:**
- Defined structure with Pydantic
- LLM filled it out
- Automatic validation
- Can now use in code: `result.is_relevant`

---

## Part 3: Complete System

Now let's build the full procurement system.

**Architecture:** Tender → Filter → Rate → Generate → Done

In [ ]:
# All imports
from pydantic import BaseModel, Field
from typing import List, Optional
from enum import Enum
from datetime import datetime
import asyncio
import httpx
import json

In [ ]:
# === DATA MODELS ===

class Category(str, Enum):
    CYBER = "cybersecurity"
    AI = "ai"
    SOFTWARE = "software"
    OTHER = "other"

class Tender(BaseModel):
    id: str
    title: str
    description: str
    organization: str
    estimated_value: Optional[str] = None

class FilterResult(BaseModel):
    is_relevant: bool
    confidence: float = Field(ge=0, le=1)
    categories: List[Category]
    reasoning: str

class RatingResult(BaseModel):
    overall_score: float = Field(ge=0, le=10)
    strategic_fit: float = Field(ge=0, le=10)
    win_probability: float = Field(ge=0, le=10)
    strengths: List[str]
    risks: List[str]
    recommendation: str

class BidDoc(BaseModel):
    executive_summary: str
    technical_approach: str
    value_proposition: str

print("✅ Models defined")

In [ ]:
# === LLM SERVICE ===

class LLM:
    def __init__(self):
        self.url = "http://localhost:1234/v1"
        self.model = "llama-3.1-8b-instruct"
    
    async def generate(self, prompt: str, schema: BaseModel, 
                       system: str = "", temp: float = 0.1) -> BaseModel:
        messages = []
        if system:
            messages.append({"role": "system", "content": system})
        
        schema_json = json.dumps(schema.model_json_schema(), indent=2)
        full_prompt = f"""{prompt}

Respond ONLY with valid JSON matching:
{schema_json}"""
        
        messages.append({"role": "user", "content": full_prompt})
        
        async with httpx.AsyncClient(timeout=120) as client:
            r = await client.post(
                f"{self.url}/chat/completions",
                json={
                    "model": self.model,
                    "messages": messages,
                    "temperature": temp,
                    "max_tokens": 2000,
                }
            )
            content = r.json()["choices"][0]["message"]["content"]
        
        # Clean
        clean = content.strip()
        for marker in ["```json", "```"]:
            clean = clean.replace(marker, "")
        
        return schema.model_validate_json(clean.strip())

llm = LLM()
print("✅ LLM ready")

### Agent 1: Filter

In [ ]:
class FilterAgent:
    def __init__(self, llm: LLM):
        self.llm = llm
    
    async def filter(self, tender: Tender) -> FilterResult:
        prompt = f"""Analyze tender:

TITLE: {tender.title}
DESC: {tender.description}

RELEVANT if: cybersecurity, AI/ML, or software development
NOT relevant if: hardware, construction, non-tech services
"""
        return await self.llm.generate(
            prompt, FilterResult,
            system="You are a procurement analyst. Be precise.",
            temp=0.1
        )

print("✅ Filter agent defined")

### Agent 2: Rater

In [ ]:
class RatingAgent:
    def __init__(self, llm: LLM):
        self.llm = llm
    
    async def rate(self, tender: Tender, cats: List[str]) -> RatingResult:
        prompt = f"""Rate opportunity:

TENDER: {tender.title}
VALUE: {tender.estimated_value}
CATEGORIES: {', '.join(cats)}

Score (0-10):
- Strategic fit
- Win probability
- Overall score

List 3 strengths, 3 risks, give recommendation.
"""
        return await self.llm.generate(
            prompt, RatingResult,
            system="You are a business analyst. Be realistic.",
            temp=0.1
        )

print("✅ Rating agent defined")

### Agent 3: Doc Generator

In [ ]:
class DocGenerator:
    def __init__(self, llm: LLM):
        self.llm = llm
    
    async def generate(self, tender: Tender, cats: List[str], 
                       strengths: List[str]) -> BidDoc:
        prompt = f"""Create bid document:

TENDER: {tender.title}
CLIENT: {tender.organization}
OUR EXPERTISE: {', '.join(cats)}
STRENGTHS: {', '.join(strengths)}

Write:
1. Executive summary (2-3 paragraphs)
2. Technical approach (2-3 paragraphs)
3. Value proposition (2 paragraphs)

Professional, specific, compelling.
"""
        return await self.llm.generate(
            prompt, BidDoc,
            system="You are an expert proposal writer.",
            temp=0.7  # Higher for creativity
        )

print("✅ Doc generator defined")

### Orchestrator

In [ ]:
class Orchestrator:
    def __init__(self):
        self.llm = LLM()
        self.filter = FilterAgent(self.llm)
        self.rater = RatingAgent(self.llm)
        self.doc_gen = DocGenerator(self.llm)
    
    async def process(self, tender: Tender):
        print(f"\n{'='*60}")
        print(f"Processing: {tender.title[:50]}...")
        print(f"{'='*60}")
        
        start = datetime.now()
        
        # Step 1: Filter
        print("\n[1/3] Filtering...")
        filter_result = await self.filter.filter(tender)
        print(f"  Relevant: {filter_result.is_relevant} ({filter_result.confidence:.0%})")
        print(f"  Categories: {[c.value for c in filter_result.categories]}")
        
        if not filter_result.is_relevant or filter_result.confidence < 0.6:
            print("\n  → Skipping (not relevant)")
            return None
        
        # Step 2: Rate
        print("\n[2/3] Rating...")
        cats = [c.value for c in filter_result.categories]
        rating = await self.rater.rate(tender, cats)
        print(f"  Score: {rating.overall_score:.1f}/10")
        print(f"  Win Prob: {rating.win_probability:.1f}/10")
        
        if rating.overall_score < 7.0:
            print("\n  → Skipping docs (score < 7.0)")
            return {"filter": filter_result, "rating": rating}
        
        # Step 3: Generate
        print("\n[3/3] Generating document...")
        doc = await self.doc_gen.generate(tender, cats, rating.strengths)
        print("  ✅ Document ready")
        
        elapsed = (datetime.now() - start).total_seconds()
        print(f"\n  Time: {elapsed:.1f}s")
        
        return {
            "filter": filter_result,
            "rating": rating,
            "document": doc,
            "time": elapsed
        }

orchestrator = Orchestrator()
print("✅ Orchestrator ready")

---

## Test Data

In [ ]:
# Sample tenders
tenders = [
    Tender(
        id="T001",
        title="AI-Powered Cybersecurity Platform",
        description="""Develop AI-based threat detection system for government 
        infrastructure. Must use machine learning for anomaly detection and 
        integrate with existing SIEM tools.""",
        organization="National Cyber Agency",
        estimated_value="€3.2M"
    ),
    Tender(
        id="T002",
        title="Office Furniture Supply",
        description="""Supply ergonomic office furniture for 500 workstations 
        including desks, chairs, and storage.""",
        organization="Ministry of Public Works",
        estimated_value="€450K"
    ),
    Tender(
        id="T003",
        title="Custom Healthcare CRM System",
        description="""Build cloud-based CRM for healthcare network. Must 
        handle patient data, appointments, GDPR compliant, mobile app included.""",
        organization="Regional Health Authority",
        estimated_value="€1.8M"
    ),
]

print(f"✅ {len(tenders)} test tenders ready")

---

## Run the System!

In [ ]:
# Process all tenders
results = []

for tender in tenders:
    result = await orchestrator.process(tender)
    results.append({"tender": tender, "result": result})

---

## Results Analysis

In [ ]:
print("\n\n" + "="*60)
print("SUMMARY")
print("="*60 + "\n")

relevant = sum(1 for r in results if r["result"] and "filter" in r["result"])
high_scored = sum(1 for r in results if r["result"] and "rating" in r["result"] 
                  and r["result"]["rating"].overall_score >= 7.0)
docs_made = sum(1 for r in results if r["result"] and "document" in r["result"])

print(f"Total Tenders: {len(tenders)}")
print(f"Relevant: {relevant}")
print(f"High Scored (≥7.0): {high_scored}")
print(f"Documents Generated: {docs_made}")

total_time = sum(r["result"]["time"] for r in results if r["result"] and "time" in r["result"])
print(f"\nTotal Time: {total_time:.1f}s")
print(f"Avg per Tender: {total_time/len(tenders):.1f}s")

### Detailed Results

In [ ]:
for i, item in enumerate(results, 1):
    tender = item["tender"]
    result = item["result"]
    
    print(f"\n{'─'*60}")
    print(f"TENDER {i}: {tender.title}")
    print(f"{'─'*60}")
    
    if not result:
        print("Status: Filtered out")
        continue
    
    if "filter" in result:
        f = result["filter"]
        print(f"Relevant: {f.is_relevant} ({f.confidence:.0%})")
        print(f"Categories: {[c.value for c in f.categories]}")
    
    if "rating" in result:
        r = result["rating"]
        print(f"\nScore: {r.overall_score:.1f}/10")
        print(f"Fit: {r.strategic_fit:.1f} | Win: {r.win_probability:.1f}")
        print(f"\nStrengths:")
        for s in r.strengths:
            print(f"  • {s}")
        print(f"\nRecommendation: {r.recommendation[:100]}...")
    
    if "document" in result:
        print(f"\n✅ Document generated")
        print(f"\nExecutive Summary:")
        print(result["document"].executive_summary[:200] + "...")

---

## 🎓 What You Just Built

Congratulations! You just built a complete AI agent system that:

1. **Filters** tenders by relevance (classification)
2. **Rates** opportunities on multiple dimensions (scoring)
3. **Generates** professional documents (content creation)
4. **Orchestrates** multiple agents in a workflow

**Key Patterns You Learned:**
- LLM API calls (HTTP requests)
- Structured outputs (Pydantic schemas)
- Prompt engineering (clear instructions)
- Temperature control (precision vs creativity)
- Multi-agent design (sequential workflow)

---

## 🧪 Experiments to Try

**1. Temperature Impact:**
- Change temperature in FilterAgent to 0.5
- Run again, compare confidence scores
- Does it change decisions?

**2. Prompt Engineering:**
- Modify FilterAgent prompt to be more specific
- Add examples of relevant tenders
- Does accuracy improve?

**3. Add Your Own Tender:**
- Create a new Tender object
- Process it through the system
- Analyze the results

**4. Scoring Threshold:**
- Change the 7.0 threshold in orchestrator
- Try 5.0 or 8.0
- How does it affect document generation?

---

## 📝 Next Steps

**Notebook 02:** Experiments with prompt engineering  
**Notebook 03:** Real data collection & validation  
**Notebook 04:** Production code structure  

---

## 💾 Save Your Work

```bash
# In terminal:
git add learn/01_mvp_complete.ipynb
git commit -m "Complete: First working AI agent system

Built 3-agent orchestration:
- Filter: 95% confidence on test cases
- Rating: Multi-dimensional scoring
- Generator: Professional document creation

Processing time: ~8s per tender on M1 Mac
Cost: $0 (local LLM)

Ready for experimentation phase."

git tag -a v0.1-mvp -m "First working MVP complete"
```